# AILib Tutorial 6: Chains - Sequential Operations

Chains allow you to connect multiple operations in sequence, creating powerful workflows. In this tutorial, you'll learn:

- Creating and running basic chains
- Passing data between chain steps
- Building complex multi-step workflows
- Error handling in chains
- Async chain operations
- Real-world chain patterns

## Setup

Let's import what we need:

In [ ]:
from ailib import OpenAIClient
from ailib.chains import Chain
from ailib.prompts import PromptTemplate
from dotenv import load_dotenv
import json

# Load environment variables
load_dotenv()

# Create a client
client = OpenAIClient()
print("Ready to build chains!")

## Basic Chain Usage

Chains connect operations that run in sequence:

In [ ]:
# Create a simple chain
chain = Chain()

# Add steps to the chain
chain.add_step(
    "generate_idea",
    lambda: client.complete("Generate a creative business idea for a mobile app")
)

chain.add_step(
    "analyze_feasibility",
    lambda idea: client.complete(f"Analyze the feasibility of this idea: {idea}")
)

chain.add_step(
    "create_pitch",
    lambda analysis: client.complete(f"Create a 30-second elevator pitch based on: {analysis}")
)

# Run the chain
result = chain.run()

print("Chain Result:")
print(result)
print("\n" + "="*50 + "\n")

# Access intermediate results
print("Intermediate Results:")
for step_name, step_result in chain.results.items():
    print(f"\n{step_name}:")
    print(step_result[:150] + "...")

## Fluent API

Chains support a fluent API for cleaner code:

In [ ]:
# Build a chain using fluent API
result = (Chain()
    .add_step("topic", lambda: "artificial intelligence in healthcare")
    .add_step("research", lambda topic: client.complete(
        f"List 3 current challenges in {topic}"
    ))
    .add_step("solutions", lambda research: client.complete(
        f"Based on these challenges: {research}\n\nPropose innovative solutions."
    ))
    .add_step("summary", lambda solutions: client.complete(
        f"Summarize these solutions in 2 sentences: {solutions}"
    ))
    .run()
)

print("Final Summary:")
print(result)

## Chains with Templates

Combine chains with prompt templates for more control:

In [ ]:
# Define templates for each step
analysis_template = PromptTemplate(
    """Analyze this text for {aspect}:
    
    Text: {text}
    
    Provide a detailed analysis focusing on {focus}."""
)

improvement_template = PromptTemplate(
    """Based on this analysis: {analysis}
    
    Suggest {num_suggestions} specific improvements.
    Format as a numbered list."""
)

# Create a text improvement chain
def create_text_improvement_chain(text, aspect="clarity", focus="readability"):
    return (Chain()
        .add_step("analyze", lambda: client.complete(
            analysis_template.format(
                text=text,
                aspect=aspect,
                focus=focus
            )
        ))
        .add_step("improve", lambda analysis: client.complete(
            improvement_template.format(
                analysis=analysis,
                num_suggestions=3
            )
        ))
        .add_step("apply", lambda suggestions: client.complete(
            f"Apply these improvements to the original text:\n{suggestions}\n\nOriginal: {text}"
        ))
    )

# Use the chain
sample_text = "The implementation of the new system will be done by the team soon."
improvement_chain = create_text_improvement_chain(sample_text, "clarity", "specificity")
improved_text = improvement_chain.run()

print("Original Text:")
print(sample_text)
print("\nImproved Text:")
print(improved_text)

## Data Transformation Chains

Chains can transform data through multiple steps:

In [ ]:
# Create a data processing chain
def create_data_chain():
    return Chain()

# Step 1: Generate raw data
def generate_data():
    response = client.complete(
        """Generate a JSON array of 5 products with fields: 
        name, price, category, rating. 
        Make it realistic e-commerce data."""
    )
    # Extract JSON from response
    try:
        # Find JSON in response
        import re
        json_match = re.search(r'\[.*\]', response, re.DOTALL)
        if json_match:
            return json.loads(json_match.group())
    except:
        pass
    return [{"name": "Sample", "price": 10, "category": "test", "rating": 4}]

# Step 2: Filter products
def filter_products(products):
    # Filter products with rating >= 4
    filtered = [p for p in products if p.get('rating', 0) >= 4]
    return filtered

# Step 3: Analyze filtered data
def analyze_products(products):
    analysis_prompt = f"""Analyze these products and provide insights:
    {json.dumps(products, indent=2)}
    
    Include: average price, popular categories, and recommendations."""
    return client.complete(analysis_prompt)

# Step 4: Generate report
def generate_report(analysis):
    report_prompt = f"""Convert this analysis into a professional business report:
    {analysis}
    
    Format with sections: Executive Summary, Key Findings, Recommendations."""
    return client.complete(report_prompt)

# Build and run the chain
data_chain = (Chain()
    .add_step("generate", generate_data)
    .add_step("filter", filter_products)
    .add_step("analyze", analyze_products)
    .add_step("report", generate_report)
)

report = data_chain.run()
print("Generated Report:")
print(report[:500] + "...")

# Show intermediate data
print("\n" + "="*50 + "\n")
print("Filtered Products:")
print(json.dumps(data_chain.results.get('filter', []), indent=2))

## Error Handling in Chains

Implement robust error handling:

In [ ]:
# Create a chain with error handling
class SafeChain(Chain):
    """Chain with built-in error handling."""
    
    def __init__(self, on_error="continue"):
        super().__init__()
        self.on_error = on_error  # "continue", "stop", or callable
        self.errors = {}
    
    def run(self, initial_input=None):
        """Run chain with error handling."""
        result = initial_input
        
        for step_name, step_func in self.steps:
            try:
                # Run step
                if result is None:
                    result = step_func()
                else:
                    result = step_func(result)
                
                # Store result
                self.results[step_name] = result
                
            except Exception as e:
                # Store error
                self.errors[step_name] = str(e)
                
                # Handle error based on policy
                if self.on_error == "stop":
                    raise
                elif self.on_error == "continue":
                    result = f"Error in {step_name}: {str(e)}"
                elif callable(self.on_error):
                    result = self.on_error(step_name, e, result)
        
        return result

# Test error handling
def risky_operation(x):
    if "error" in x.lower():
        raise ValueError("Operation failed!")
    return f"Processed: {x}"

# Create chain with error recovery
safe_chain = SafeChain(on_error="continue")
safe_chain.add_step("step1", lambda: "Starting data")
safe_chain.add_step("step2", lambda x: x + " -> adding more")
safe_chain.add_step("step3", lambda x: risky_operation(x + " ERROR"))  # This will fail
safe_chain.add_step("step4", lambda x: f"Final: {x}")

result = safe_chain.run()
print("Chain completed with result:")
print(result)
print("\nErrors encountered:")
print(safe_chain.errors)

# Custom error handler
def custom_error_handler(step_name, error, last_result):
    print(f"Handling error in {step_name}: {error}")
    return f"Recovered from {step_name} error"

custom_chain = SafeChain(on_error=custom_error_handler)
custom_chain.add_step("risky", lambda: risky_operation("This has ERROR in it"))
custom_chain.add_step("safe", lambda x: f"Continued after: {x}")

print("\n" + "="*50 + "\n")
print("Custom error handling:")
result = custom_chain.run()
print(f"Final result: {result}")

## Conditional Chains

Create chains with conditional logic:

In [ ]:
class ConditionalChain(Chain):
    """Chain with conditional branching."""
    
    def __init__(self):
        super().__init__()
        self.conditions = {}
    
    def add_conditional_step(self, name, condition, true_func, false_func=None):
        """Add a step that runs based on a condition."""
        def conditional_wrapper(input_data):
            if condition(input_data):
                return true_func(input_data)
            elif false_func:
                return false_func(input_data)
            else:
                return input_data
        
        self.add_step(name, conditional_wrapper)
        return self

# Create a content moderation chain
def create_moderation_chain():
    chain = ConditionalChain()
    
    # Step 1: Analyze content
    chain.add_step(
        "analyze",
        lambda text: {
            "text": text,
            "sentiment": client.complete(f"Analyze sentiment of: {text}. Reply with just: positive, negative, or neutral"),
            "length": len(text)
        }
    )
    
    # Step 2: Conditional processing based on sentiment
    chain.add_conditional_step(
        "process_sentiment",
        condition=lambda data: "negative" in data["sentiment"].lower(),
        true_func=lambda data: {
            **data,
            "action": "review",
            "message": client.complete(f"Rewrite this in a more positive tone: {data['text']}")
        },
        false_func=lambda data: {
            **data,
            "action": "approve",
            "message": data["text"]
        }
    )
    
    # Step 3: Format response
    chain.add_step(
        "format",
        lambda data: f"""Content Analysis:
- Original: {data['text']}
- Sentiment: {data['sentiment']}
- Action: {data['action']}
- Final Message: {data['message']}"""
    )
    
    return chain

# Test with different inputs
moderation_chain = create_moderation_chain()

test_texts = [
    "This product is terrible and I hate it!",
    "I love this service, it's amazing!",
    "The weather is okay today."
]

for text in test_texts:
    print(f"\nProcessing: '{text}'")
    print("-" * 50)
    result = moderation_chain.run(text)
    print(result)

## Parallel Chains

Run multiple chains in parallel and combine results:

In [ ]:
class ParallelChain:
    """Run multiple chains in parallel."""
    
    def __init__(self):
        self.chains = {}
    
    def add_chain(self, name, chain):
        """Add a chain to run in parallel."""
        self.chains[name] = chain
        return self
    
    def run(self, input_data=None):
        """Run all chains and collect results."""
        results = {}
        
        # In production, use asyncio or threading
        for name, chain in self.chains.items():
            results[name] = chain.run(input_data)
        
        return results

# Create parallel analysis chains
def create_analysis_chains(text):
    # Technical analysis chain
    technical_chain = (Chain()
        .add_step("complexity", lambda: client.complete(
            f"Analyze technical complexity of: {text}. Rate 1-10 with explanation."
        ))
        .add_step("jargon", lambda complexity: client.complete(
            f"Identify technical jargon in: {text}"
        ))
    )
    
    # Readability chain
    readability_chain = (Chain()
        .add_step("grade_level", lambda: client.complete(
            f"What reading grade level is this text: {text}"
        ))
        .add_step("simplify", lambda level: client.complete(
            f"Simplify this text for general audience: {text}"
        ))
    )
    
    # Sentiment chain
    sentiment_chain = (Chain()
        .add_step("emotion", lambda: client.complete(
            f"Identify emotions in: {text}"
        ))
        .add_step("tone", lambda emotion: client.complete(
            f"Describe the overall tone of: {text}"
        ))
    )
    
    return ParallelChain()
        .add_chain("technical", technical_chain)
        .add_chain("readability", readability_chain)
        .add_chain("sentiment", sentiment_chain)

# Run parallel analysis
sample_text = """The quantum entanglement phenomenon demonstrates non-local correlations 
between particles, challenging our classical understanding of reality."""

parallel_analysis = create_analysis_chains(sample_text)
results = parallel_analysis.run()

print("Parallel Analysis Results:")
print("=" * 50)
for analysis_type, result in results.items():
    print(f"\n{analysis_type.upper()} Analysis:")
    print(result[:200] + "...")

## Chain Composition

Compose complex chains from simpler ones:

In [ ]:
# Create reusable chain components
class ChainLibrary:
    """Library of reusable chain components."""
    
    @staticmethod
    def translation_chain(target_language):
        """Create a translation chain."""
        return Chain().add_step(
            "translate",
            lambda text: client.complete(f"Translate to {target_language}: {text}")
        )
    
    @staticmethod
    def summary_chain(length="brief"):
        """Create a summarization chain."""
        return Chain().add_step(
            "summarize",
            lambda text: client.complete(f"Create a {length} summary of: {text}")
        )
    
    @staticmethod
    def format_chain(format_type):
        """Create a formatting chain."""
        return Chain().add_step(
            "format",
            lambda text: client.complete(f"Format this as {format_type}: {text}")
        )
    
    @staticmethod
    def compose(*chains):
        """Compose multiple chains into one."""
        composed = Chain()
        
        for i, chain in enumerate(chains):
            for step_name, step_func in chain.steps:
                # Rename steps to avoid conflicts
                composed.add_step(f"{i}_{step_name}", step_func)
        
        return composed

# Create a multi-language announcement chain
announcement = "We're launching a new AI-powered feature next week!"

# Compose chains for different purposes
multilingual_chain = ChainLibrary.compose(
    Chain().add_step("original", lambda: announcement),
    ChainLibrary.summary_chain("very brief"),
    ChainLibrary.translation_chain("Spanish"),
    ChainLibrary.format_chain("tweet")
)

result = multilingual_chain.run()
print("Multilingual Tweet:")
print(result)

# Create a document processing pipeline
print("\n" + "="*50 + "\n")

document = """Artificial Intelligence is transforming industries worldwide. 
From healthcare to finance, AI applications are improving efficiency and decision-making. 
However, ethical considerations and job displacement remain significant challenges."""

processing_pipeline = ChainLibrary.compose(
    Chain().add_step("input", lambda: document),
    ChainLibrary.summary_chain("one sentence"),
    ChainLibrary.format_chain("professional email subject line")
)

email_subject = processing_pipeline.run()
print("Email Subject Line:")
print(email_subject)

## Real-World Example: Content Pipeline

Build a complete content processing pipeline:

In [ ]:
class ContentPipeline:
    """Complete content processing pipeline."""
    
    def __init__(self, client):
        self.client = client
    
    def create_blog_pipeline(self):
        """Create a blog post generation pipeline."""
        return (Chain()
            # Research phase
            .add_step("research", self._research_topic)
            .add_step("outline", self._create_outline)
            .add_step("draft", self._write_draft)
            
            # Enhancement phase
            .add_step("enhance", self._enhance_content)
            .add_step("seo", self._optimize_seo)
            
            # Finalization phase
            .add_step("format", self._format_post)
            .add_step("metadata", self._generate_metadata)
        )
    
    def _research_topic(self, topic):
        """Research the topic."""
        return self.client.complete(
            f"Research key points about '{topic}'. List 5 important aspects."
        )
    
    def _create_outline(self, research):
        """Create post outline."""
        return self.client.complete(
            f"""Based on this research: {research}
            
            Create a blog post outline with:
            - Introduction
            - 3-4 main sections
            - Conclusion"""
        )
    
    def _write_draft(self, outline):
        """Write the first draft."""
        return self.client.complete(
            f"""Write a 500-word blog post based on this outline:
            {outline}
            
            Make it engaging and informative."""
        )
    
    def _enhance_content(self, draft):
        """Enhance the content."""
        return self.client.complete(
            f"""Enhance this blog post by:
            - Adding compelling examples
            - Improving transitions
            - Making it more engaging
            
            Post: {draft}"""
        )
    
    def _optimize_seo(self, content):
        """Optimize for SEO."""
        return {
            "content": content,
            "keywords": self.client.complete(
                f"Extract 5 SEO keywords from: {content[:200]}..."
            )
        }
    
    def _format_post(self, data):
        """Format the final post."""
        return {
            **data,
            "formatted": self.client.complete(
                f"Format this blog post with proper headings and sections: {data['content']}"
            )
        }
    
    def _generate_metadata(self, data):
        """Generate post metadata."""
        return {
            **data,
            "title": self.client.complete(
                f"Create a catchy title for this post: {data['content'][:200]}..."
            ),
            "meta_description": self.client.complete(
                f"Write a 150-character meta description for: {data['content'][:200]}..."
            ),
            "tags": self.client.complete(
                f"Suggest 5 tags for this post. Return as comma-separated list: {data['content'][:200]}..."
            )
        }

# Use the content pipeline
pipeline = ContentPipeline(client)
blog_chain = pipeline.create_blog_pipeline()

# Generate a blog post
topic = "The Future of Remote Work"
result = blog_chain.run(topic)

print(f"Blog Post Generation Complete!")
print("=" * 50)
print(f"\nTitle: {result.get('title', 'N/A')}")
print(f"\nMeta Description: {result.get('meta_description', 'N/A')}")
print(f"\nTags: {result.get('tags', 'N/A')}")
print(f"\nKeywords: {result.get('keywords', 'N/A')}")
print(f"\nContent Preview:")
print(result.get('formatted', result.get('content', ''))[:500] + "...")

## Best Practices

Tips for building effective chains:

In [ ]:
# 1. Keep steps focused and single-purpose
good_chain = (Chain()
    .add_step("parse", lambda x: x.split(","))
    .add_step("clean", lambda x: [item.strip() for item in x])
    .add_step("filter", lambda x: [item for item in x if item])
)

# 2. Use descriptive step names
descriptive_chain = (Chain()
    .add_step("fetch_user_data", lambda id: f"User data for {id}")
    .add_step("validate_permissions", lambda data: f"Validated: {data}")
    .add_step("generate_report", lambda data: f"Report: {data}")
)

# 3. Handle data transformation explicitly
def transform_chain():
    def parse_response(response):
        """Parse LLM response into structured data."""
        # Add error handling
        try:
            # Parse logic here
            return {"parsed": response}
        except Exception as e:
            return {"error": str(e), "raw": response}
    
    return (Chain()
        .add_step("generate", lambda: client.complete("Generate data"))
        .add_step("parse", parse_response)
        .add_step("validate", lambda x: x if "error" not in x else None)
    )

# 4. Create reusable chain factories
def create_validation_chain(rules):
    """Factory for creating validation chains."""
    chain = Chain()
    
    for i, rule in enumerate(rules):
        chain.add_step(f"validate_{i}", rule)
    
    return chain

# 5. Log and monitor chain execution
class MonitoredChain(Chain):
    def run(self, initial_input=None):
        print(f"Starting chain with {len(self.steps)} steps")
        start_time = datetime.now()
        
        result = super().run(initial_input)
        
        duration = (datetime.now() - start_time).total_seconds()
        print(f"Chain completed in {duration:.2f} seconds")
        
        return result

print("Best practices examples created!")

## Summary

In this tutorial, you learned:

- ✅ How to create and run basic chains
- ✅ How to pass data between chain steps
- ✅ How to use the fluent API for cleaner code
- ✅ How to combine chains with templates
- ✅ How to handle errors in chains
- ✅ How to create conditional and parallel chains
- ✅ How to compose complex chains from simple ones
- ✅ Best practices for chain design

Chains are powerful for:
- Building multi-step workflows
- Processing data through multiple transformations
- Creating reusable processing pipelines
- Orchestrating complex AI operations

## Next Steps

Continue learning with:

- **Tutorial 7: Tools and Decorators** - Extend agent capabilities
- **Tutorial 8: Agents** - Build autonomous AI agents
- **Tutorial 9: Advanced Features** - Async operations and more

Happy chaining! 🔗